# Finding the optimum spot for the Halewijn Award Ceremony in Haarlem, the Netherlands. 
#### Applied Data Science Capstone Project by IBM/Coursera
   
## *CRW Korver*, Haarlem, the Netherlands

December 15, 2019 \
email author: <crwkorver@hotmail.com>
***

## Table of contents
* [Introduction](#1)
* [Data](#2)
* [Methodology](#3)
* [Exploratory Data Analysis](#4)
* [Results and Discussion](#5)
* [Conclusion](#6)

***



## 1. Introduction <a name="1"></a>

The [**Halewijn Prijs**](https://nl.wikipedia.org/wiki/Halewijnprijs,_literatuurprijs_van_de_stad_Roermond) is a Dutch national award, intended for literary talent that, based on quality and irresistibility of his or her published work, deserves extra interest. The prize consists of a cash prize and a bronze small statue made by Dick van Wijk. 

The yearly Award Ceremony will take place in the city of Haarlem, the Netherlands. To attain maximal public attendance, the ideal neighborhood would be nearby the railway station in order to facilitate traveling by public means in a car-crowded city like Haarlem. Furthermore, to maximize potential book sales, a location should be found that is in the near vicinity of book shops around. Finally, ample restaurants should be present here as well, to accommodate the audience (and the winner and jury members, who will dine together afterwards).



## 2. Data <a name="2"></a>

In order to meet the above mentioned criteria, we should gather insight in 
* existing neighborhoods of Haarlem, based on the Dutch Postal Code system;
* the number of existing bookshops in the various neighborhoods of Haarlem;
* the number of restaurants in the neighborhood, if any.

Delineation of neighborhoods will be based on the Dutch Postal Code dictionary.

### 2.1 Data sources

The following data sources were used to extract the information required:
* **Nederlandse Postcodetabel** > Dutch Postal Code Dictionary in Excel xlsx format ([documentation](http://www.sqlblog.nl/postcodetabel-nederland-sql-script/)): postal codes and coordinates.\
NB: A Dutch Postal Code API is [available](https://www.postcodeapi.nu/?gclid=EAIaIQobChMIpPeUz86P5gIVC553Ch2RcwO5EAAYASAAEgLMEvD_BwE), but only as a paid service.
* **GeoPy** > a Python 3 client for third-party geocoding web services: coordinates of Haarlem Railway Station. ([documentation](https://geopy.readthedocs.io/en/stable//))
* **Foursquare API** > location data of book shops and restaurants. ([documentation](https://developer.foursquare.com/docs))

### 2.2 Data cleaning and Feature selection

From the primary Dutch postal codes dataset pd_Dutchpc (n=471781; 46.8Mb), all non-Haarlem records were dropped. From the resulting dataset (n=4257) PO Box data (all with one geographical location) were discarded, leaving 4123 records. Inspection of the dataframe revealed the presence of (10) missing values in the column PostcodeLetters. However, this column, as well as 7 other columns, were not informative with regard to current analysis, so they were dropped. Then street names were aggregated into unique postal codes, but still 4032 entries remained. Therefore, unique postal codes were further segmented into 19 different Neighborhoods \[Wijk\] (based on the Postal Code \[Postcodenummer\]). The features selected thus were: Neighborhood, Street Names, averaged latitudes and longitudes.

Inventory of dataframes used:
- **df_Dutchpc** = raw data of all Dutch postal codes (n=471781; 13 columns; 46.8 Mb)
- df_HaarlemPCLatM = temporary dataframe containing averaged latitudes for each postal code
- df_HaarlemPCLonM = temporary dataframe containing averaged longitudes for each postal code
- **df_Haarlem01** = all Haarlem postal codes, essential columns only (n=4123; 5 columns; 161.2 Kb)
- df_Haarlem02 = temporary dataframe containing unique Haarlem postal codes, street names aggregated (3 columns)
- **df_Haarlem03** = unique Haarlem postal codes, street names aggregated, latitude&longitude averaged (n=4032; 5 diff.columns; 157.6 Kb)
- df_Haarlem04 = temporary dataframe containing unique Haarlem neighborhoods, street names aggregated (2 columns)
- df_HaarlemNLatM = temporary dataframe containing averaged latitudes for each neighborhood
- df_HaarlemNLonM = temporary dataframe containing averaged longitudes for each neighborhood
- **df_Haarlem05** = all Haarlem neighborhoods, essential columns only (n=19; 4 columns; 0.736 Kb)
- **df_HrlCluster** = definitive dataset, containing Haarlem neighborhoods, geodata, number and relative frequencies of nearby book stores and restaurants (n=19; 4 columns; 1.4 Kb)

Inventory of generated maps:
- Haarlem01_map = primary Haarlem map, centered around Station
- Haarlem02_map = 01 with Central Station and neighborhoods superimposed
- Haarlem03_map = 02 with book shops nearby superimposed
- Haarlem04_map = 03 with restaurants nearby superimposed
- HrlCluster_map = 01 with clustered neighborhoods indicated



## 3. Methodology <a name="3"></a>

Will be presented in a separate report.


## 4. Exploratory Data Analysis <a name="4"></a>

### 4.1 Construction of the primary data set

* Importing necessary libraries

In [1]:
import numpy as np # data handling in a vectorized manner
import pandas as pd # data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
!pip install xlrd # data extraction from Excel & conversion to pd dataframe
import json # handling JSON files
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # address conversion into coordinates
import requests # requests handling
from pandas.io.json import json_normalize # tranforming JSON file into pandas dataframe
import matplotlib.cm as cm # data plotting
import matplotlib.colors as colors
from sklearn.cluster import KMeans # cluster analysis
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering

     |████████████████████████████████| 112kB 31.0MB/s eta 0:00:01
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    scipy-1.3.2                |   py36h921218d_0        18.0 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    liblapacke-3.8

* Importing primary dataset

Primary postal code Excel file was dowloaded as zip-file, extracted and stored locally in Skills Network Labs as postcodetabel.xlsx. \
This file contained 471781 records! \
To spare computational resources, this file was stripped to 4257 Haarlem-only records and stored as Haarlempostcodetabel.xlsx file

* Inspection of the dataset

In [2]:
path = 'Haarlempostcodetabel.xlsx'
df_Haarlem01 = pd.read_excel(path)
df_Haarlem01.head()

,PostcodeID,PostCodePK,PostCode,PostcodeNummers,PostcodeLetters,Straat,MinNummer,MaxNummer,Plaats,Gemeente,Provincie,Latitude,Longitude
0,258150,2000AA_0,2000AA,2000,AA,Postbus,0,10000,Haarlem,Haarlem,Noord-Holland,52.381016,4.64567
1,430279,2000AB_0,2000AB,2000,AB,Postbus,0,10000,Haarlem,Haarlem,Noord-Holland,52.381016,4.64567
2,79515,2000AC_0,2000AC,2000,AC,Postbus,0,10000,Haarlem,Haarlem,Noord-Holland,52.381016,4.64567
3,88048,2000AD_0,2000AD,2000,AD,Postbus,0,10000,Haarlem,Haarlem,Noord-Holland,52.381016,4.64567
4,282603,2000AE_0,2000AE,2000,AE,Postbus,0,10000,Haarlem,Haarlem,Noord-Holland,52.381016,4.64567


In [3]:
df_Haarlem01.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4257 entries, 0 to 4256
Data columns (total 13 columns):
PostcodeID         4257 non-null int64
PostCodePK         4257 non-null object
PostCode           4257 non-null object
PostcodeNummers    4257 non-null int64
PostcodeLetters    4246 non-null object
Straat             4257 non-null object
MinNummer          4257 non-null int64
MaxNummer          4257 non-null int64
Plaats             4257 non-null object
Gemeente           4257 non-null object
Provincie          4257 non-null object
Latitude           4257 non-null float64
Longitude          4257 non-null float64
dtypes: float64(2), int64(4), object(7)
memory usage: 432.5+ KB


### 4.2 Data cleaning and Feature selection

a) the raw Haarlem dataset

In [4]:
print('Number of records in the Haarlem-only dataset:',df_Haarlem01.shape[0])

Number of records in the Haarlem-only dataset: 4257


In [5]:
# 4.2.2 'Postbus' (PO Box) values in column Straat are not informative (all with same geolocation)
df_Haarlem01 = df_Haarlem01[df_Haarlem01.Straat != 'Postbus']
df_Haarlem01.reset_index(drop=True,inplace=True)
print('Remaining records in the dataset:',df_Haarlem01.shape[0])

Remaining records in the dataset: 4123


In [6]:
# 4.2.3 Are there any missing values?
missing = df_Haarlem01.isnull()
for column in missing.columns.values.tolist():
  print(column)
  print(missing[column].value_counts())

PostcodeID
False    4123
Name: PostcodeID, dtype: int64
PostCodePK
False    4123
Name: PostCodePK, dtype: int64
PostCode
False    4123
Name: PostCode, dtype: int64
PostcodeNummers
False    4123
Name: PostcodeNummers, dtype: int64
PostcodeLetters
False    4113
True       10
Name: PostcodeLetters, dtype: int64
Straat
False    4123
Name: Straat, dtype: int64
MinNummer
False    4123
Name: MinNummer, dtype: int64
MaxNummer
False    4123
Name: MaxNummer, dtype: int64
Plaats
False    4123
Name: Plaats, dtype: int64
Gemeente
False    4123
Name: Gemeente, dtype: int64
Provincie
False    4123
Name: Provincie, dtype: int64
Latitude
False    4123
Name: Latitude, dtype: int64
Longitude
False    4123
Name: Longitude, dtype: int64


*feature selection*

In [7]:
# Yes, column PostcodeLetters is the only one containing (10) missing values
# However, for our analysis we only need the columns PostCode, PostcodeNummers, Straat, Latitude and Longitude
df_Haarlem01.drop(['PostcodeID','PostCodePK','PostcodeLetters','MinNummer','MaxNummer','Plaats','Gemeente','Provincie'],axis=1,inplace=True)
df_Haarlem01.head()

,PostCode,PostcodeNummers,Straat,Latitude,Longitude
0,2011AA,2011,Spaarnwouderstraat,52.379617,4.641031
1,2011AA,2011,Melkboersteeg,52.381016,4.645670
2,2011AB,2011,Sleutelstraat,52.380279,4.644150
3,2011AB,2011,Spaarnwouderstraat,52.380131,4.643988
4,2011AC,2011,Koralensteeg,52.380790,4.645376


b) Haarlem Unique postal codes

In [8]:
# 4.2.4 Aggregate streets based on unique Postcode in a new dataframe
df_Haarlem02 = df_Haarlem01.groupby(['PostCode','PostcodeNummers'])['Straat'].apply(lambda x: "%s" % ', '.join(x)).reset_index()
df_Haarlem02.rename(columns={'Straat':'StraatNamen'},inplace=True)
print('Remaining records in the dataset:',df_Haarlem02.shape[0])

Remaining records in the dataset: 4032


In [9]:
df_Haarlem02.head()

,PostCode,PostcodeNummers,StraatNamen
0,2011AA,2011,"Spaarnwouderstraat, Melkboersteeg"
1,2011AB,2011,"Sleutelstraat, Spaarnwouderstraat"
2,2011AC,2011,"Koralensteeg, Spaarnwouderstraat"
3,2011AD,2011,"Spaarnwouderstraat, Wijdesteeg"
4,2011AE,2011,"Ossenhoofdsteeg, Wijdesteeg, Spaarnwouderstraat"


In [10]:
# Average latitude and longitude, pertaining to each unique postal code
df_HaarlemPCLatM = df_Haarlem01.groupby('PostCode', as_index=False)['Latitude'].mean()
df_HaarlemPCLatM.head()

,PostCode,Latitude
0,2011AA,52.380317
1,2011AB,52.380205
2,2011AC,52.380699
3,2011AD,52.379602
4,2011AE,52.379657


In [11]:
df_HaarlemPCLonM = df_Haarlem01.groupby('PostCode', as_index=False)['Longitude'].mean()
df_HaarlemPCLonM.head()

,PostCode,Longitude
0,2011AA,4.643351
1,2011AB,4.644069
2,2011AC,4.645494
3,2011AD,4.641006
4,2011AE,4.642028


In [12]:
# These 3 datasets have corresponding indices (i.e. sorted on PostCode)
# Check whether the datasets also have the same size
len(df_Haarlem02) == len(df_HaarlemPCLatM) == len(df_HaarlemPCLonM)

True

In [13]:
# So we can add the average latitude & longitude columns to a new dataframe
df_Haarlem03 = df_Haarlem02
df_Haarlem03['Latitude_m']=df_HaarlemPCLatM['Latitude']
df_Haarlem03['Longitude_m']=df_HaarlemPCLonM['Longitude']

In [14]:
df_Haarlem03.head()

,PostCode,PostcodeNummers,StraatNamen,Latitude_m,Longitude_m
0,2011AA,2011,"Spaarnwouderstraat, Melkboersteeg",52.380317,4.643351
1,2011AB,2011,"Sleutelstraat, Spaarnwouderstraat",52.380205,4.644069
2,2011AC,2011,"Koralensteeg, Spaarnwouderstraat",52.380699,4.645494
3,2011AD,2011,"Spaarnwouderstraat, Wijdesteeg",52.379602,4.641006
4,2011AE,2011,"Ossenhoofdsteeg, Wijdesteeg, Spaarnwouderstraat",52.379657,4.642028


In [15]:
df_Haarlem03.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4032 entries, 0 to 4031
Data columns (total 5 columns):
PostCode           4032 non-null object
PostcodeNummers    4032 non-null int64
StraatNamen        4032 non-null object
Latitude_m         4032 non-null float64
Longitude_m        4032 non-null float64
dtypes: float64(2), int64(1), object(2)
memory usage: 157.6+ KB


c) Haarlem Unique neighborhoods

In [16]:
# A valuable dataset. However, still 4032 entries left > aggregate into neighborhoods
# How many neighborhoods are present
print("Number of unique neighborhoods:",df_Haarlem03['PostcodeNummers'].nunique())

Number of unique neighborhoods: 19


In [17]:
# Which ones?
df_Haarlem03['PostcodeNummers'].unique()

array([2011, 2012, 2013, 2014, 2015, 2019, 2021, 2022, 2023, 2024, 2025,
       2026, 2031, 2032, 2033, 2034, 2035, 2036, 2037])

In [18]:
# Create a new dataframe with 2 columns
df_Haarlem04 = df_Haarlem03
df_Haarlem04.drop('PostCode',axis=1,inplace=True)

In [19]:
df_Haarlem04 = df_Haarlem04.groupby('PostcodeNummers')['StraatNamen'].apply(lambda x: "%s" % ', '.join(x)).reset_index()
df_Haarlem04.rename(columns={'PostcodeNummers':'Wijk'},inplace=True)

In [20]:
df_Haarlem04.head()

,Wijk,StraatNamen
0,2011,"Spaarnwouderstraat, Melkboersteeg, Sleutelstra..."
1,2012,"Zuider Buiten Spaarne, Zuider Buiten Spaarne, ..."
2,2013,"Duvenvoordestraat, Duvenvoordestraat, Duvenvoo..."
3,2014,"Willem Bontekoestraat, Houtvaartpad, Houtmanka..."
4,2015,"Tulpenkade, Krokusstraat, Krokusstraat, Krokus..."


In [21]:
# Average latitude and longitude, pertaining to each neighborhood
df_HaarlemNLatM = df_Haarlem03.groupby('PostcodeNummers', as_index=False)['Latitude_m'].mean()
df_HaarlemNLatM.head()

,PostcodeNummers,Latitude_m
0,2011,52.381240
1,2012,52.370950
2,2013,52.382397
3,2014,52.373490
4,2015,52.381513


In [22]:
df_HaarlemNLonM = df_Haarlem03.groupby('PostcodeNummers', as_index=False)['Longitude_m'].mean()
df_HaarlemNLonM.head()

,PostcodeNummers,Longitude_m
0,2011,4.635808
1,2012,4.629562
2,2013,4.624165
3,2014,4.614884
4,2015,4.609336


In [23]:
len(df_Haarlem04) == len(df_HaarlemNLatM) == len(df_HaarlemNLonM)

True

In [24]:
# Add the renewed average latitude & longitude columns into a new dataframe
df_Haarlem05 = df_Haarlem04
df_Haarlem05['Latitude_m']=df_HaarlemNLatM['Latitude_m']
df_Haarlem05['Longitude_m']=df_HaarlemNLonM['Longitude_m']

In [25]:
df_Haarlem05.head()

,Wijk,StraatNamen,Latitude_m,Longitude_m
0,2011,"Spaarnwouderstraat, Melkboersteeg, Sleutelstra...",52.381240,4.635808
1,2012,"Zuider Buiten Spaarne, Zuider Buiten Spaarne, ...",52.370950,4.629562
2,2013,"Duvenvoordestraat, Duvenvoordestraat, Duvenvoo...",52.382397,4.624165
3,2014,"Willem Bontekoestraat, Houtvaartpad, Houtmanka...",52.373490,4.614884
4,2015,"Tulpenkade, Krokusstraat, Krokusstraat, Krokus...",52.381513,4.609336


In [26]:
# Change datatype of Wijk from integer to object
df_Haarlem05.Wijk = df_Haarlem05.Wijk.astype(str) # necessary to show labels on the map
df_Haarlem05.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 4 columns):
Wijk           19 non-null object
StraatNamen    19 non-null object
Latitude_m     19 non-null float64
Longitude_m    19 non-null float64
dtypes: float64(2), object(2)
memory usage: 736.0+ bytes


### 4.3 Building primary map overview

In [27]:
# Geographical coordinates
address = 'Haarlem'
geolocator = Nominatim(user_agent="hrl_explorer")
location = geolocator.geocode(address)
Hrl_lat = location.latitude
Hrl_lon = location.longitude
print('Geographical coordinates of {}: {}N, {}E.'.format(address, Hrl_lat, Hrl_lon))

Geographical coordinates of Haarlem: 52.38370575N, 4.64355969655963E.


In [28]:
# However, our survey is centered around Haarlem Central Station
address = 'Haarlem Stationsplein'
geolocator = Nominatim(user_agent="hrl_explorer")
location = geolocator.geocode(address)
HrlSt_lat = location.latitude
HrlSt_lon = location.longitude
print('Coordinates of Haarlem Station: {}N, {}E.'.format(HrlSt_lat, HrlSt_lon))

Coordinates of Haarlem Station: 52.3880778N, 4.6385788E.


In [29]:
# create the map
Haarlem01_map = folium.Map(location=[HrlSt_lat,HrlSt_lon], zoom_start=13)
Haarlem01_map

In [30]:
folium.features.CircleMarker([HrlSt_lat, HrlSt_lon],radius=10,color= 'red',popup= 'STATION',fill = False,fill_color = 'red',fill_opacity = 0.6).add_to(Haarlem01_map)
Haarlem01_map

In [31]:
# instantiate a feature group for Haarlem neighborhoods in the dataframe
Haarlem02_map = Haarlem01_map
Haarlem_Neighbor = folium.map.FeatureGroup()
# loop through all 19 neighborhoods and add each to the Haarlem_Neighbor feature group
for lat, lng, label in zip(df_Haarlem05.Latitude_m, df_Haarlem05.Longitude_m, df_Haarlem05.Wijk):
    Haarlem_Neighbor.add_child(folium.features.CircleMarker([lat, lng],radius=5, popup=label, color='grey',fill=True,fill_color='grey',fill_opacity=1.0))
    label = folium.Popup(label, parse_html=True)
# add neighborhoods to map
Haarlem02_map.add_child(Haarlem_Neighbor)

In [32]:
from geopy.distance import geodesic

In [33]:
# Examining the distance between neighborhood centers by varying value of a and b
a = 1 # recordA nr in dataframe
b = 14 # recordB nr in dataframe
A_name = df_Haarlem05.loc[a, 'Wijk']
A_ll = (df_Haarlem05.loc[a, 'Latitude_m'],df_Haarlem05.loc[a, 'Longitude_m'])
B_name = df_Haarlem05.loc[b, 'Wijk']
B_ll = (df_Haarlem05.loc[b, 'Latitude_m'],df_Haarlem05.loc[b, 'Longitude_m'])
print('The distance between neighborhoods', A_name, 'and', B_name, 'is',int(geodesic(A_ll, B_ll).m),'m.')

The distance between neighborhoods 2012 and 2033 is 1905 m.


Visual inspection of the Haarlem map & running the above code revealed the top 5 distances between nearby neighborhood centers:
* 2012-2033: 1905 m 
* 2012-2032: 1792 m
* 2019-2023: 1769 m
* 2013-2023: 1717 m
* 2031-2022: 1472 m

Therefore, a radius of 1000 m was implemented in assigning features to a particular neighborhood using Foursquare (see 4.5). \
Greatest distances between neighborhood centers are: 2026-2036: 7459 m, 2026-2037: 6988 m, 2025-2036: 6943 m, 2025-2037: 6499 m, 2026-2034: 6390 m. \
Minimum distance between two postal code neighborhoods was 57 m (2011-2019). 

### 4.4 Creating definite map of Haarlem with feature (location) data from Foursquare

In [34]:
# 4.3.1 Foursquare credentials
ID = 'WNIALDO3G2WR10S3ZQYQR2ZSX03JJKOXMAUOOPP0AV2ZNFYZ' # Foursquare ID
SECRET = '0VG3P0L1YLVHOVER103MD5A15OOZ0W2VLHJQQKPK2354IC5M' # Foursquare Secret
version = '20191201'
limit = 30
print('Foursquare credentials are hidden')

Foursquare credentials are hidden


**a) book stores**

In [35]:
# 4.3.2 Searching for book shops within 2.5 km of Haarlem Station (with latitude = HrlSt_lat & longitude = HrlSt_lon)
search = 'boek'
radius = 2500
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ID, SECRET, HrlSt_lat, HrlSt_lon, version, search, radius, limit)

In [36]:
# Send the GET request
HrlBook_result = requests.get(url).json()
HrlBook_result

{'meta': {'code': 200, 'requestId': '5df0eb8e542890001bd30ae0'},
 'response': {'venues': [{'id': '4b925f13f964a52013f533e3',
    'name': 'H. de Vries Boeken',
    'location': {'address': 'Gedempte Oude Gracht 27',
     'lat': 52.381420134616484,
     'lng': 4.632425604352738,
     'labeledLatLngs': [{'label': 'display',
       'lat': 52.381420134616484,
       'lng': 4.632425604352738}],
     'distance': 850,
     'postalCode': '2011 GK',
     'cc': 'NL',
     'city': 'Haarlem',
     'state': 'Noord-Holland',
     'country': 'Nederland',
     'formattedAddress': ['Gedempte Oude Gracht 27',
      '2011 GK Haarlem',
      'Nederland']},
    'categories': [{'id': '4bf58dd8d48988d114951735',
      'name': 'Bookstore',
      'pluralName': 'Bookstores',
      'shortName': 'Bookstore',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/bookstore_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1576070035',
    'hasPerk': False},
   {'id': '4da03f5

In [37]:
# assign relevant data of JSON file to Haarlem_Books & convert into dataframe df_HrlBook01
Haarlem_Books = HrlBook_result['response']['venues'] # returns all metadata
df_HrlBook01 = json_normalize(Haarlem_Books) # convert it to a dataframe
df_HrlBook01.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,4b925f13f964a52013f533e3,H. de Vries Boeken,"[{'id': '4bf58dd8d48988d114951735', 'name': 'B...",v-1576070035,False,Gedempte Oude Gracht 27,52.381420,4.632426,"[{'label': 'display', 'lat': 52.38142013461648...",850,2011 GK,NL,Haarlem,Noord-Holland,Nederland,"[Gedempte Oude Gracht 27, 2011 GK Haarlem, Ned..."
1,4da03f5d58c2224b2ec04d79,Kennemer Boekhandel,"[{'id': '4bf58dd8d48988d114951735', 'name': 'B...",v-1576070035,False,Kleverparkweg 3,52.390937,4.631174,"[{'label': 'display', 'lat': 52.39093693293498...",595,NaN,NL,Haarlem,Noord-Holland,Nederland,"[Kleverparkweg 3, Haarlem, Nederland]"
2,4cf3a02f7e0da1cd82b69d97,Boekenvoordeel,"[{'id': '4bf58dd8d48988d114951735', 'name': 'B...",v-1576070035,False,Grote Houtstraat 108,52.378020,4.632227,"[{'label': 'display', 'lat': 52.37802, 'lng': ...",1199,2011 ST,NL,Haarlem,Noord-Holland,Nederland,"[Grote Houtstraat 108, 2011 ST Haarlem, Nederl..."
3,4c419a4daf052d7f61907d79,Athenaeum Boekhandel,"[{'id': '4bf58dd8d48988d114951735', 'name': 'B...",v-1576070035,False,Gedempte Oude Gracht 70,52.380104,4.632651,"[{'label': 'display', 'lat': 52.38010449085371...",974,2011 GT,NL,Haarlem,Noord-Holland,Nederland,"[Gedempte Oude Gracht 70, 2011 GT Haarlem, Ned..."
4,4c04d2630d0e0f47cd6a049a,Plantage boekhandel,"[{'id': '4bf58dd8d48988d114951735', 'name': 'B...",v-1576070035,False,Barteljorisstraat,52.381835,4.635317,"[{'label': 'display', 'lat': 52.38183468, 'lng...",729,NaN,NL,Haarlem,Noord-Holland,Nederland,"[Barteljorisstraat, Haarlem, Nederland]"


In [38]:
# retain the book store (venue) names, and features associated with location
# create a new dataframe df_HrlBook02
f_columns = ['name', 'categories'] + [col for col in df_HrlBook01.columns if col.startswith('location.')] + ['id'] # columns we need in dataframe
df_HrlBook02 = df_HrlBook01.loc[:, f_columns] # append these columns at end
# function to extract category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
# filter the category for each row
df_HrlBook02['categories'] = df_HrlBook02.apply(get_category_type, axis=1)
# clean column names by keeping only last term
df_HrlBook02.columns = [column.split('.')[-1] for column in df_HrlBook02.columns] # everything before the point is deleted
df_HrlBook02.head(10)

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,H. de Vries Boeken,Bookstore,Gedempte Oude Gracht 27,52.381420,4.632426,"[{'label': 'display', 'lat': 52.38142013461648...",850,2011 GK,NL,Haarlem,Noord-Holland,Nederland,"[Gedempte Oude Gracht 27, 2011 GK Haarlem, Ned...",4b925f13f964a52013f533e3
1,Kennemer Boekhandel,Bookstore,Kleverparkweg 3,52.390937,4.631174,"[{'label': 'display', 'lat': 52.39093693293498...",595,NaN,NL,Haarlem,Noord-Holland,Nederland,"[Kleverparkweg 3, Haarlem, Nederland]",4da03f5d58c2224b2ec04d79
2,Boekenvoordeel,Bookstore,Grote Houtstraat 108,52.378020,4.632227,"[{'label': 'display', 'lat': 52.37802, 'lng': ...",1199,2011 ST,NL,Haarlem,Noord-Holland,Nederland,"[Grote Houtstraat 108, 2011 ST Haarlem, Nederl...",4cf3a02f7e0da1cd82b69d97
3,Athenaeum Boekhandel,Bookstore,Gedempte Oude Gracht 70,52.380104,4.632651,"[{'label': 'display', 'lat': 52.38010449085371...",974,2011 GT,NL,Haarlem,Noord-Holland,Nederland,"[Gedempte Oude Gracht 70, 2011 GT Haarlem, Ned...",4c419a4daf052d7f61907d79
4,Plantage boekhandel,Bookstore,Barteljorisstraat,52.381835,4.635317,"[{'label': 'display', 'lat': 52.38183468, 'lng...",729,NaN,NL,Haarlem,Noord-Holland,Nederland,"[Barteljorisstraat, Haarlem, Nederland]",4c04d2630d0e0f47cd6a049a
5,Boekenhoek,None,NaN,52.375111,4.644473,"[{'label': 'display', 'lat': 52.375111, 'lng':...",1497,NaN,NL,Haarlem,Noord-Holland,Nederland,"[Haarlem, Nederland]",4dfb4092e4cd5679360841c7
6,Boekhandel Bloemendaal,Bookstore,NaN,52.402331,4.618351,"[{'label': 'display', 'lat': 52.40233083637072...",2098,NaN,NL,NaN,NaN,Nederland,[Nederland],4be2a58d660ec9286dd8c93b
7,Gillissen & Co Boekhandel,Bookstore,Rijksstraatweg 125,52.403764,4.647474,"[{'label': 'display', 'lat': 52.4037642, 'lng'...",1847,2024 DC,NL,Haarlem,Noord-Holland,Nederland,"[Rijksstraatweg 125, 2024 DC Haarlem, Nederland]",54cc1d86498e7db4b0aeae0d
8,'t Boekendaaltje,Bookstore,NaN,52.402767,4.618723,"[{'label': 'display', 'lat': 52.40276718139648...",2119,NaN,NL,NaN,NaN,Nederland,[Nederland],53d8eae6498ee8ce2303326e
9,Nautilus Boekbinderij,Miscellaneous Shop,Edisonstraat 30,52.368450,4.614320,"[{'label': 'display', 'lat': 52.36845, 'lng': ...",2737,2014 KJ,NL,Haarlem,Noord-Holland,Nederland,"[Edisonstraat 30, 2014 KJ Haarlem, Nederland]",561678ed498eb55f38cfbbdf


In [39]:
df_HrlBook02.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 14 columns):
name                10 non-null object
categories          9 non-null object
address             7 non-null object
lat                 10 non-null float64
lng                 10 non-null float64
labeledLatLngs      10 non-null object
distance            10 non-null int64
postalCode          5 non-null object
cc                  10 non-null object
city                8 non-null object
state               8 non-null object
country             10 non-null object
formattedAddress    10 non-null object
id                  10 non-null object
dtypes: float64(2), int64(1), object(11)
memory usage: 1.2+ KB


In [40]:
print("Number of nearby book stores:",df_HrlBook02.shape[0])
df_HrlBook02.name

Number of nearby book stores: 10


0           H. de Vries Boeken
1          Kennemer Boekhandel
2               Boekenvoordeel
3         Athenaeum Boekhandel
4          Plantage boekhandel
5                   Boekenhoek
6       Boekhandel Bloemendaal
7    Gillissen & Co Boekhandel
8             't Boekendaaltje
9        Nautilus Boekbinderij
Name: name, dtype: object

In [41]:
# Visualize the book stores on a new map Haarlem03
Haarlem03_map = Haarlem02_map
# instantiate a feature group for Haarlem book stores in the dataframe
Haarlem_Bookstore = folium.map.FeatureGroup()
# loop through all book shops and add each to the Haarlem_Bookstore feature group
for lat, lng, lab in zip(df_HrlBook02.lat, df_HrlBook02.lng, df_HrlBook02.name):
    label = folium.Popup(lab, parse_html=True)
    Haarlem_Bookstore.add_child(folium.features.CircleMarker([lat, lng],radius=6, popup=label, color='red', fill=True,fill_color='orange',fill_opacity=0.6))
# add book stores to map
Haarlem03_map.add_child(Haarlem_Bookstore)
Haarlem03_map

**b) restaurants**

In [42]:
# 4.3.3 Searching for restaurants within 2.5 km of Haarlem Station
search = 'restaurant'
radius = 2500
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ID, SECRET, HrlSt_lat, HrlSt_lon, version, search, radius, limit)
HrlRest_result = requests.get(url).json()
Haarlem_Restaurants = HrlRest_result['response']['venues']
df_HrlRest01 = json_normalize(Haarlem_Restaurants)

f_columns = ['name', 'categories'] + [col for col in df_HrlRest01.columns if col.startswith('location.')] + ['id']
df_HrlRest02 = df_HrlRest01.loc[:, f_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

df_HrlRest02['categories'] = df_HrlRest02.apply(get_category_type, axis=1)
df_HrlRest02.columns = [column.split('.')[-1] for column in df_HrlRest02.columns]
df_HrlRest02.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,crossStreet,id
0,Café-Bar-Restaurant Beijneshal,Sports Bar,Stationsplein 134,52.387498,4.638173,"[{'label': 'display', 'lat': 52.38749846617937...",70,2011 LN,NL,Haarlem,Noord-Holland,Nederland,"[Stationsplein 134, 2011 LN Haarlem, Nederland]",NaN,NaN,4cd038907b6854810430c0f8
1,Hotel-Restaurant Carillon,Restaurant,Grote Markt 27,52.381419,4.637040,"[{'label': 'display', 'lat': 52.38141908009712...",748,2011 RC,NL,Haarlem,Noord-Holland,Nederland,"[Grote Markt 27, 2011 RC Haarlem, Nederland]",NaN,NaN,4b06d293f964a520e0f022e3
2,Restaurant ML,Restaurant,Kleine Houtstraat 70 BG,52.380780,4.638396,"[{'label': 'display', 'lat': 52.38078, 'lng': ...",812,2011 DR,NL,Haarlem,Noord-Holland,Nederland,"[Kleine Houtstraat 70 BG, 2011 DR Haarlem, Ned...",Centrum,NaN,4b76f1f8f964a5202f6d2ee3
3,Restaurant Parck,French Restaurant,Frederikspark 2,52.373977,4.630622,"[{'label': 'display', 'lat': 52.37397731042772...",1660,2012 DA,NL,Haarlem,Noord-Holland,Nederland,"[Frederikspark 2, 2012 DA Haarlem, Nederland]",NaN,NaN,4ed3a287e3007feb813e6d0a
4,Restaurant Delphi,Greek Restaurant,NaN,52.378146,4.636744,"[{'label': 'display', 'lat': 52.37814617848480...",1112,NaN,NL,Haarlem,Noord-Holland,Nederland,"[Haarlem, Nederland]",NaN,NaN,4d52c1d63062a1cd55d974a3


In [43]:
print("Number of nearby restaurants:",df_HrlRest02.shape[0])
df_HrlRest02.name

Number of nearby restaurants: 30


0           Café-Bar-Restaurant Beijneshal
1                Hotel-Restaurant Carillon
2                            Restaurant ML
3                         Restaurant Parck
4                        Restaurant Delphi
5                     Ma Browns Restaurant
6                     Restaurant Dubrovnik
7                         Restaurant Ludic
8       Chinees Restaurant de Gouden Lelie
9                          IKEA Restaurant
10                  Restaurant De Generaal
11                 Restaurant de Wandelaar
12                     Restaurant Applause
13                    Restaurant Esperanto
14                          Restaurant Ohm
15         restaurant van der Valk Haarlem
16                      Restaurant Babbels
17                  Restaurant Het Pakhuis
18                       Restaurant Scampi
19                         Restaurant Noor
20                      Restaurant Sin Yue
21    Restaurant Kunstijsbaan Kennemerland
22                       Restaurant Variee
23         

In [44]:
# Visualization
Haarlem04_map = Haarlem03_map
Haarlem_Restaurant = folium.map.FeatureGroup()
for lat, lng, lab in zip(df_HrlRest02.lat, df_HrlRest02.lng, df_HrlRest02.name):
    label = folium.Popup(lab, parse_html=True)
    Haarlem_Restaurant.add_child(folium.features.CircleMarker([lat, lng],radius=6, popup=label, color='blue', fill=True,fill_color='blue',fill_opacity=0.6))
Haarlem04_map.add_child(Haarlem_Restaurant)
Haarlem04_map

### 4.5 Exploring the 19 neighborhoods of Haarlem ###
Assigning book stores and restaurants in Haarlem to postal code areas (neighborhoods) by means of Foursquare

In [45]:
# This is current Haarlem dataset with Neighborhoods (Wijk), street names (StraatNamen) and averaged latitudes & longitudes
df_Haarlem05.head()

,Wijk,StraatNamen,Latitude_m,Longitude_m
0,2011,"Spaarnwouderstraat, Melkboersteeg, Sleutelstra...",52.381240,4.635808
1,2012,"Zuider Buiten Spaarne, Zuider Buiten Spaarne, ...",52.370950,4.629562
2,2013,"Duvenvoordestraat, Duvenvoordestraat, Duvenvoo...",52.382397,4.624165
3,2014,"Willem Bontekoestraat, Houtvaartpad, Houtmanka...",52.373490,4.614884
4,2015,"Tulpenkade, Krokusstraat, Krokusstraat, Krokus...",52.381513,4.609336


**a) Searching the first neighborhood in Haarlem**

a1) for book stores

In [46]:
# Search for book stores within radius in first neighborhood 2011
wijk_lat = df_Haarlem05.loc[0, 'Latitude_m']
wijk_lon = df_Haarlem05.loc[0, 'Longitude_m']
wijk_name = df_Haarlem05.loc[0, 'Wijk']
version = '20191201'
query = 'boek'
radius = 1000 # based on maximum nearby neighborhood distance 1905 m
limit = 30
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ID, SECRET, wijk_lat, wijk_lon, version, query, radius, limit)
wijkbook_result = requests.get(url).json()
Wijk_Books = wijkbook_result['response']['venues']

In [47]:
df_Wijkbook = json_normalize(Wijk_Books)

In [48]:
f_columns = ['id','name', 'location.address', 'location.lat', 'location.lng', 'location.distance']
df_Wijkbook2011 = df_Wijkbook.loc[:, f_columns]
df_Wijkbook2011.columns = [col.split(".")[-1] for col in df_Wijkbook2011.columns]
df_Wijkbook2011.head()

,id,name,address,lat,lng,distance
0,4c04d2630d0e0f47cd6a049a,Plantage boekhandel,Barteljorisstraat,52.381835,4.635317,74
1,4b925f13f964a52013f533e3,H. de Vries Boeken,Gedempte Oude Gracht 27,52.381420,4.632426,230
2,4c419a4daf052d7f61907d79,Athenaeum Boekhandel,Gedempte Oude Gracht 70,52.380104,4.632651,249
3,4cf3a02f7e0da1cd82b69d97,Boekenvoordeel,Grote Houtstraat 108,52.378020,4.632227,433
4,4dfb4092e4cd5679360841c7,Boekenhoek,NaN,52.375111,4.644473,901


In [49]:
print('# book stores in neighborhood {} within radius {} m:'.format(wijk_name,radius),df_Wijkbook2011.shape[0])

# book stores in neighborhood 2011 within radius 1000 m: 5


a2) for restaurants, the exercise is not repeated (see b2)

**b) Searching all neighborhoods in Haarlem**

b1) for book stores

In [50]:
version = '20191201'
query = 'boek'
radius = 1000 # based on maximum nearby neighborhood distance 1905 m
limit = 30
wijk_nrbooks = []
for x in range(19):
    wijk_lat = df_Haarlem05.loc[x, 'Latitude_m']
    wijk_lon = df_Haarlem05.loc[x, 'Longitude_m']
    wijk_name = df_Haarlem05.loc[x, 'Wijk']
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ID, SECRET, wijk_lat, wijk_lon, version, query, radius, limit)
    wijkbook_result = requests.get(url).json()
    Wijk_Books = wijkbook_result['response']['venues']
    df_Wijkbook = json_normalize(Wijk_Books)
    wijk_nrbooks.append(df_Wijkbook.shape[0])
#    print('# book stores in neighborhood {}:'.format(wijk_name),df_Wijkbook.shape[0])
print(wijk_nrbooks)

[5, 2, 5, 2, 0, 5, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]


b2) for restaurants

In [51]:
version = '20191201'
query = 'restaurant'
radius = 1000 # based on maximum nearby neighborhood distance 1905 m
limit = 30
wijk_nrrest = []
for x in range(19):
    wijk_lat = df_Haarlem05.loc[x, 'Latitude_m']
    wijk_lon = df_Haarlem05.loc[x, 'Longitude_m']
    wijk_name = df_Haarlem05.loc[x, 'Wijk']
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ID, SECRET, wijk_lat, wijk_lon, version, query, radius, limit)
    wijkrest_result = requests.get(url).json()
    Wijk_Rest = wijkrest_result['response']['venues']
    df_WijkRest = json_normalize(Wijk_Rest)
    wijk_nrrest.append(df_WijkRest.shape[0])
print(wijk_nrrest)

[30, 27, 30, 7, 10, 30, 8, 4, 4, 4, 3, 1, 1, 30, 7, 8, 5, 5, 3]


In [52]:
# create a new dataframe HrlCluster with numbers and relative frequencies of book shops & restaurants
df_HrlCluster=df_Haarlem05
df_HrlCluster['NrBook'] = wijk_nrbooks
book_totnr = df_HrlCluster['NrBook'].sum()
df_HrlCluster['RFBook'] = df_HrlCluster['NrBook']/book_totnr
df_HrlCluster['NrRest'] = wijk_nrrest
rest_totnr = df_HrlCluster['NrRest'].sum()
df_HrlCluster['RFRest'] = df_HrlCluster['NrRest']/rest_totnr

In [53]:
df_HrlCluster.head()

,Wijk,StraatNamen,Latitude_m,Longitude_m,NrBook,RFBook,NrRest,RFRest
0,2011,"Spaarnwouderstraat, Melkboersteeg, Sleutelstra...",52.381240,4.635808,5,0.161290,30,0.138249
1,2012,"Zuider Buiten Spaarne, Zuider Buiten Spaarne, ...",52.370950,4.629562,2,0.064516,27,0.124424
2,2013,"Duvenvoordestraat, Duvenvoordestraat, Duvenvoo...",52.382397,4.624165,5,0.161290,30,0.138249
3,2014,"Willem Bontekoestraat, Houtvaartpad, Houtmanka...",52.373490,4.614884,2,0.064516,7,0.032258
4,2015,"Tulpenkade, Krokusstraat, Krokusstraat, Krokus...",52.381513,4.609336,0,0.000000,10,0.046083


### 4.6 Clustering the 19 neighborhoods of Haarlem ###
Cluster postal code areas (neighborhoods) to create centers with high book store and restaurant concentrations. \
Those clusters with corresponding addresses and (relative) number of book stores and restaurants are the final result of this analysis.

In [54]:
# Clustering analysis should be performed on the relative frequencies of book stores and restaurants only
df_HrlClusterAnal = df_HrlCluster.drop(['Wijk','StraatNamen','Latitude_m','Longitude_m','NrBook','NrRest'],axis=1)

In [55]:
df_HrlClusterAnal.head()

,RFBook,RFRest
0,0.161290,0.138249
1,0.064516,0.124424
2,0.161290,0.138249
3,0.064516,0.032258
4,0.000000,0.046083


In [56]:
# K-Means clustering
k = 5 # nr of clusters
kmeans = KMeans(n_clusters=k, random_state=0).fit(df_HrlClusterAnal)
kmeans.labels_[0:19]  # cluster labels for each record

array([2, 0, 2, 1, 4, 2, 1, 3, 3, 3, 3, 3, 3, 0, 1, 1, 3, 3, 3],
      dtype=int32)

In [57]:
df_HrlCluster.insert(0, 'Cluster', kmeans.labels_)

In [58]:
df_HrlCluster.head()

,Cluster,Wijk,StraatNamen,Latitude_m,Longitude_m,NrBook,RFBook,NrRest,RFRest
0,2,2011,"Spaarnwouderstraat, Melkboersteeg, Sleutelstra...",52.381240,4.635808,5,0.161290,30,0.138249
1,0,2012,"Zuider Buiten Spaarne, Zuider Buiten Spaarne, ...",52.370950,4.629562,2,0.064516,27,0.124424
2,2,2013,"Duvenvoordestraat, Duvenvoordestraat, Duvenvoo...",52.382397,4.624165,5,0.161290,30,0.138249
3,1,2014,"Willem Bontekoestraat, Houtvaartpad, Houtmanka...",52.373490,4.614884,2,0.064516,7,0.032258
4,4,2015,"Tulpenkade, Krokusstraat, Krokusstraat, Krokus...",52.381513,4.609336,0,0.000000,10,0.046083


### 4.7 Examine clusters ###

#### Cluster 1

In [59]:
print('Cluster nr 1')
df_HrlCluster.loc[df_HrlCluster['Cluster'] == 0, df_HrlCluster.columns[[1,5,7,6,8]]]

Cluster nr 1


,Wijk,NrBook,NrRest,RFBook,RFRest
1,2012,2,27,0.064516,0.124424
13,2032,1,30,0.032258,0.138249


Cluster 1 (indicated in red on map) comprises 2 neighborhoods, with eccentric location. The number of book stores is relatively low, however lots of restaurants are present within 1000 m.

#### Cluster 2

In [60]:
print('Cluster nr 2')
df_HrlCluster.loc[df_HrlCluster['Cluster'] == 1, df_HrlCluster.columns[[1,5,7,6,8]]]

Cluster nr 2


,Wijk,NrBook,NrRest,RFBook,RFRest
3,2014,2,7,0.064516,0.032258
6,2021,1,8,0.032258,0.036866
14,2033,1,7,0.032258,0.032258
15,2034,1,8,0.032258,0.036866


Cluster 2 (purple) comprises 4 neighborhoods, widely separated from each other. The number of book stores as well as restaurants is reltively low.

#### Cluster 3

In [61]:
print('Cluster nr 3')
df_HrlCluster.loc[df_HrlCluster['Cluster'] == 2, df_HrlCluster.columns[[1,5,7,6,8]]]

Cluster nr 3


,Wijk,NrBook,NrRest,RFBook,RFRest
0,2011,5,30,0.16129,0.138249
2,2013,5,30,0.16129,0.138249
5,2019,5,30,0.16129,0.138249


Cluster 3 (blue) comprises 3 nearby neighborhoods, located in the center of Haarlem. The number of book stores and restaurants is high. \
This cluster seems promising.

#### Cluster 4

In [62]:
print('Cluster nr 4')
df_HrlCluster.loc[df_HrlCluster['Cluster'] == 3, df_HrlCluster.columns[[1,5,7,6,8]]]

Cluster nr 4


,Wijk,NrBook,NrRest,RFBook,RFRest
7,2022,1,4,0.032258,0.018433
8,2023,1,4,0.032258,0.018433
9,2024,1,4,0.032258,0.018433
10,2025,1,3,0.032258,0.013825
11,2026,1,1,0.032258,0.004608
12,2031,0,1,0.000000,0.004608
16,2035,1,5,0.032258,0.023041
17,2036,1,5,0.032258,0.023041
18,2037,1,3,0.032258,0.013825


Cluster 4 (green) comprises 9 neighborhoods, on the outborders of Haarlem. Number of book stores as well as restaurants is very low. \
This cluster should not be recommended. 

#### Cluster 5

In [63]:
print('Cluster nr 5')
df_HrlCluster.loc[df_HrlCluster['Cluster'] == 4, df_HrlCluster.columns[[1,5,7,6,8]]]

Cluster nr 5


,Wijk,NrBook,NrRest,RFBook,RFRest
4,2015,0,10,0.0,0.046083


Finally, cluster 5 (yellow) is composed of just one neighborhood, relatively far from Central Station and with no book stores present; however there are still some restaurants present. 

In [64]:
df_HrlCluster.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 9 columns):
Cluster        19 non-null int32
Wijk           19 non-null object
StraatNamen    19 non-null object
Latitude_m     19 non-null float64
Longitude_m    19 non-null float64
NrBook         19 non-null int64
RFBook         19 non-null float64
NrRest         19 non-null int64
RFRest         19 non-null float64
dtypes: float64(4), int32(1), int64(2), object(2)
memory usage: 1.4+ KB


#### Distance from Haarlem Central Station

In [65]:
# Calculating distance between neighborhood centroids and Central Station
a = 13 # recordA nr in dataframe
A_name = df_Haarlem05.loc[a, 'Wijk']
A_ll = (df_Haarlem05.loc[a, 'Latitude_m'],df_Haarlem05.loc[a, 'Longitude_m'])
B_name = 'Central Station'
B_ll = (52.3880778,4.6385788)
print('The distance between neighborhood', A_name, 'and', B_name, 'is',int(geodesic(A_ll, B_ll).m),'m.')

The distance between neighborhood 2032 and Central Station is 1410 m.


### **Visualization** ###

In [66]:
df_HrlCluster.head()

,Cluster,Wijk,StraatNamen,Latitude_m,Longitude_m,NrBook,RFBook,NrRest,RFRest
0,2,2011,"Spaarnwouderstraat, Melkboersteeg, Sleutelstra...",52.381240,4.635808,5,0.161290,30,0.138249
1,0,2012,"Zuider Buiten Spaarne, Zuider Buiten Spaarne, ...",52.370950,4.629562,2,0.064516,27,0.124424
2,2,2013,"Duvenvoordestraat, Duvenvoordestraat, Duvenvoo...",52.382397,4.624165,5,0.161290,30,0.138249
3,1,2014,"Willem Bontekoestraat, Houtvaartpad, Houtmanka...",52.373490,4.614884,2,0.064516,7,0.032258
4,4,2015,"Tulpenkade, Krokusstraat, Krokusstraat, Krokus...",52.381513,4.609336,0,0.000000,10,0.046083


In [67]:
HrlCluster_map = folium.Map(location=[HrlSt_lat,HrlSt_lon], zoom_start=13)
folium.features.CircleMarker([HrlSt_lat, HrlSt_lon],radius=10,color= 'red',popup= 'STATION',fill = False,fill_color = 'red',fill_opacity = 0.6).add_to(HrlCluster_map)
HrlCluster_map

In [68]:
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, wijk, cluster in zip(df_HrlCluster['Latitude_m'], df_HrlCluster['Longitude_m'], df_HrlCluster['Wijk'], df_HrlCluster['Cluster']):
    label = folium.Popup(str(wijk) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon],radius=5,popup=label,color=rainbow[cluster-1],fill=True,fill_color=rainbow[cluster-1],fill_opacity=0.7).add_to(HrlCluster_map)
HrlCluster_map


## Results and Discussion <a name="5"></a>

Will be presented in a separate report.


## Conclusion <a name="6"></a>

Will be presented in a separate report.